In [ ]:
"""
Advanced Test Suite for Comprehensive System Scanner

This module provides intelligent, comprehensive testing for the system scanner
including unit tests, integration tests, performance tests, and security tests.
"""

import asyncio
import json
import logging
import os
import platform
import pytest
import tempfile
import time
import unittest
from unittest.mock import Mock, patch, MagicMock, AsyncMock
from datetime import datetime, timedelta
from pathlib import Path
import psutil
import threading
from typing import Dict, List, Any

# Import the modules to test
from comprehensive_system_scanner import (
    ComprehensiveSystemScanner,
    DynamicRepairEngine,
    ProtectionEngine,
    SystemIssue,
    RepairAction,
    CommandExecutionError
)

# Configure test logging
logging.basicConfig(level=logging.DEBUG)
test_logger = logging.getLogger(__name__)

class TestSystemIssue(unittest.TestCase):
    """Test SystemIssue dataclass functionality"""

    def test_system_issue_creation(self):
        """Test creating a SystemIssue with all fields"""
        issue = SystemIssue(
            issue_id="TEST001",
            category="performance",
            severity="HIGH",
            description="High CPU usage detected",
            impact="System slowdown",
            auto_fixable=True,
            fix_commands=["taskkill /f /im process.exe"],
            manual_steps=["Check task manager"],
            prevention_measures=["Regular monitoring"]
        )

        self.assertEqual(issue.issue_id, "TEST001")
        self.assertEqual(issue.severity, "HIGH")
        self.assertTrue(issue.auto_fixable)
        self.assertEqual(len(issue.fix_commands), 1)

    def test_system_issue_defaults(self):
        """Test SystemIssue with default values"""
        issue = SystemIssue(
            issue_id="TEST002",
            category="security",
            severity="CRITICAL",
            description="Security vulnerability",
            impact="Data breach risk",
            auto_fixable=False
        )

        self.assertEqual(len(issue.fix_commands), 0)
        self.assertEqual(len(issue.manual_steps), 0)
        self.assertEqual(len(issue.prevention_measures), 0)

class TestRepairAction(unittest.TestCase):
    """Test RepairAction dataclass functionality"""

    def test_repair_action_creation(self):
        """Test creating a RepairAction with all fields"""
        action = RepairAction(
            action_id="REPAIR001",
            name="Test Repair",
            description="Test repair action",
            commands=["echo test"],
            requires_admin=False,
            estimated_time=60,
            success_criteria=["test completed"],
            rollback_commands=["echo rollback"]
        )

        self.assertEqual(action.action_id, "REPAIR001")
        self.assertFalse(action.requires_admin)
        self.assertEqual(action.estimated_time, 60)
        self.assertEqual(len(action.rollback_commands), 1)

class TestDynamicRepairEngine(unittest.IsolatedAsyncioTestCase):
    """Advanced tests for DynamicRepairEngine"""

    def setUp(self):
        """Set up test environment"""
        self.repair_engine = DynamicRepairEngine()
        self.test_repair_action = RepairAction(
            action_id="test_repair",
            name="Test Repair Action",
            description="A test repair action",
            commands=["echo 'test command'"],
            requires_admin=False,
            estimated_time=30,
            success_criteria=["test command"]
        )

    def test_repair_templates_initialization(self):
        """Test that repair templates are properly initialized"""
        templates = self.repair_engine.repair_templates

        # Check that basic templates exist
        self.assertIn("temp_cleanup", templates)
        self.assertIn("service_optimization", templates)
        self.assertIn("startup_optimization", templates)

        # Verify template structure
        temp_cleanup = templates["temp_cleanup"]
        self.assertIsInstance(temp_cleanup, RepairAction)
        self.assertTrue(len(temp_cleanup.commands) > 0)
        self.assertIsInstance(temp_cleanup.requires_admin, bool)

    @patch('subprocess.run')
    async def test_execute_command_success(self, mock_subprocess):
        """Test successful command execution"""
        # Mock successful subprocess result
        mock_result = Mock()
        mock_result.returncode = 0
        mock_result.stdout = "Command executed successfully"
        mock_result.stderr = ""
        mock_subprocess.return_value = mock_result

        result = await self.repair_engine._execute_command("echo test", False)

        self.assertTrue(result["success"])
        self.assertEqual(result["return_code"], 0)
        self.assertIn("Command executed successfully", result["stdout"])

    @patch('subprocess.run')
    async def test_execute_command_failure(self, mock_subprocess):
        """Test failed command execution"""
        # Mock failed subprocess result
        mock_result = Mock()
        mock_result.returncode = 1
        mock_result.stdout = ""
        mock_result.stderr = "Command failed"
        mock_subprocess.return_value = mock_result

        result = await self.repair_engine._execute_command("invalid_command", False)

        self.assertFalse(result["success"])
        self.assertEqual(result["return_code"], 1)
        self.assertIn("Command failed", result["stderr"])

    @patch('subprocess.run')
    async def test_execute_repair_success(self, mock_subprocess):
        """Test successful repair execution"""
        # Mock successful subprocess result
        mock_result = Mock()
        mock_result.returncode = 0
        mock_result.stdout = "test command executed"
        mock_result.stderr = ""
        mock_subprocess.return_value = mock_result

        progress_callback = Mock()
        result = await self.repair_engine.execute_repair(
            self.test_repair_action,
            progress_callback
        )

        self.assertTrue(result["success"])
        self.assertIn("repair_id", result)
        self.assertIn("duration", result)

        # Verify progress callback was called
        self.assertTrue(progress_callback.called)

    def test_check_success_criteria_with_criteria(self):
        """Test success criteria checking with specific criteria"""
        results = [
            {"stdout": "Operation completed successfully", "stderr": "", "success": True},
            {"stdout": "All tests passed", "stderr": "", "success": True}
        ]
        criteria = ["completed successfully", "tests passed"]

        success = self.repair_engine._check_success_criteria(results, criteria)
        self.assertTrue(success)

    def test_check_success_criteria_without_criteria(self):
        """Test success criteria checking without specific criteria"""
        results = [
            {"success": True},
            {"success": True}
        ]
        criteria = []

        success = self.repair_engine._check_success_criteria(results, criteria)
        self.assertTrue(success)

    def test_check_success_criteria_failure(self):
        """Test success criteria checking with failure"""
        results = [
            {"stdout": "Operation failed", "stderr": "Error occurred", "success": False}
        ]
        criteria = ["completed successfully"]

        success = self.repair_engine._check_success_criteria(results, criteria)
        self.assertFalse(success)

    async def test_repair_progress_tracking(self):
        """Test repair progress tracking functionality"""
        repair_id = "test_repair_123"

        # Test progress update
        await self.repair_engine._update_repair_progress(
            repair_id, "Starting repair", 0, "running"
        )

        # Since repair_id doesn't exist, it should log a warning
        # This tests the error handling in progress tracking

    @patch('subprocess.run')
    async def test_rollback_functionality(self, mock_subprocess):
        """Test repair rollback functionality"""
        # Create repair action with rollback commands
        repair_action = RepairAction(
            action_id="rollback_test",
            name="Rollback Test",
            description="Test rollback functionality",
            commands=["failing_command"],
            requires_admin=False,
            estimated_time=30,
            success_criteria=["success"],
            rollback_commands=["echo 'rolling back'"]
        )

        # Mock failing command followed by successful rollback
        mock_results = [
            Mock(returncode=1, stdout="", stderr="Command failed"),  # Failing command
            Mock(returncode=0, stdout="rolling back", stderr="")     # Successful rollback
        ]
        mock_subprocess.side_effect = mock_results

        repair_id = "rollback_test_123"

        # This should trigger rollback due to command failure
        with self.assertRaises(Exception):
            await self.repair_engine._execute_repair_action(
                repair_action, None, repair_id
            )

class TestProtectionEngine(unittest.TestCase):
    """Advanced tests for ProtectionEngine"""

    def setUp(self):
        """Set up test environment"""
        with patch('threading.Thread'):  # Prevent actual thread creation during tests
            self.protection_engine = ProtectionEngine()

    def test_protection_rules_initialization(self):
        """Test protection rules initialization"""
        rules = self.protection_engine.protection_rules

        self.assertIn("file_integrity", rules)
        self.assertIn("registry_protection", rules)

        # Check file integrity rules
        file_rules = rules["file_integrity"]
        self.assertIn("monitor_paths", file_rules)
        self.assertIn("check_interval", file_rules)
        self.assertIsInstance(file_rules["monitor_paths"], list)

    def test_threat_database_loading(self):
        """Test threat database loading"""
        threat_db = self.protection_engine.threat_database

        # Should have basic structure even if file doesn't exist
        self.assertIsInstance(threat_db, dict)

    @patch('os.path.exists')
    @patch('builtins.open')
    def test_threat_database_from_file(self, mock_open, mock_exists):
        """Test loading threat database from file"""
        mock_exists.return_value = True
        mock_file_content = {
            "malicious_hashes": ["abc123", "def456"],
            "suspicious_domains": ["malware.com"],
            "last_updated": "2023-01-01T00:00:00"
        }
        mock_open.return_value.__enter__.return_value.read.return_value = json.dumps(mock_file_content)

        with patch('threading.Thread'):
            engine = ProtectionEngine()

        # The threat database should contain the mocked data
        self.assertIsInstance(engine.threat_database, dict)

    def test_file_hash_calculation(self):
        """Test file hash calculation"""
        # Create a temporary file for testing
        with tempfile.NamedTemporaryFile(mode='w', delete=False) as temp_file:
            temp_file.write("test content for hashing")
            temp_file_path = temp_file.name

        try:
            file_hash = self.protection_engine._calculate_file_hash(temp_file_path)
            self.assertIsInstance(file_hash, str)
            self.assertEqual(len(file_hash), 64)  # SHA-256 hash length
        finally:
            os.unlink(temp_file_path)

    def test_file_hash_calculation_nonexistent_file(self):
        """Test file hash calculation for non-existent file"""
        file_hash = self.protection_engine._calculate_file_hash("nonexistent_file.txt")
        self.assertIsNone(file_hash)

: 

In [ ]:
"""
Advanced Test Suite for Comprehensive System Scanner

This module provides intelligent, comprehensive testing for the system scanner
including unit tests, integration tests, performance tests, and security tests.
"""

import asyncio
import json
import logging
import os
import platform
import pytest
import tempfile
import time
import unittest
from unittest.mock import Mock, patch, MagicMock, AsyncMock
from datetime import datetime, timedelta
from pathlib import Path
import psutil
import threading
from typing import Dict, List, Any

# Import the modules to test
from comprehensive_system_scanner import (
    ComprehensiveSystemScanner,
    DynamicRepairEngine,
    ProtectionEngine,
    SystemIssue,
    RepairAction,
    CommandExecutionError
)

# Configure test logging
logging.basicConfig(level=logging.DEBUG)
test_logger = logging.getLogger(__name__)

class TestSystemIssue(unittest.TestCase):
    """Test SystemIssue dataclass functionality"""

    def test_system_issue_creation(self):
        """Test creating a SystemIssue with all fields"""
        issue = SystemIssue(
            issue_id="TEST001",
            category="performance",
            severity="HIGH",
            description="High CPU usage detected",
            impact="System slowdown",
            auto_fixable=True,
            fix_commands=["taskkill /f /im process.exe"],
            manual_steps=["Check task manager"],
            prevention_measures=["Regular monitoring"]
        )

        self.assertEqual(issue.issue_id, "TEST001")
        self.assertEqual(issue.severity, "HIGH")
        self.assertTrue(issue.auto_fixable)
        self.assertEqual(len(issue.fix_commands), 1)

    def test_system_issue_defaults(self):
        """Test SystemIssue with default values"""
        issue = SystemIssue(
            issue_id="TEST002",
            category="security",
            severity="CRITICAL",
            description="Security vulnerability",
            impact="Data breach risk",
            auto_fixable=False
        )

        self.assertEqual(len(issue.fix_commands), 0)
        self.assertEqual(len(issue.manual_steps), 0)
        self.assertEqual(len(issue.prevention_measures), 0)

class TestRepairAction(unittest.TestCase):
    """Test RepairAction dataclass functionality"""

    def test_repair_action_creation(self):
        """Test creating a RepairAction with all fields"""
        action = RepairAction(
            action_id="REPAIR001",
            name="Test Repair",
            description="Test repair action",
            commands=["echo test"],
            requires_admin=False,
            estimated_time=60,
            success_criteria=["test completed"],
            rollback_commands=["echo rollback"]
        )

        self.assertEqual(action.action_id, "REPAIR001")
        self.assertFalse(action.requires_admin)
        self.assertEqual(action.estimated_time, 60)
        self.assertEqual(len(action.rollback_commands), 1)

class TestDynamicRepairEngine(unittest.IsolatedAsyncioTestCase):
    """Advanced tests for DynamicRepairEngine"""

    def setUp(self):
        """Set up test environment"""
        self.repair_engine = DynamicRepairEngine()
        self.test_repair_action = RepairAction(
            action_id="test_repair",
            name="Test Repair Action",
            description="A test repair action",
            commands=["echo 'test command'"],
            requires_admin=False,
            estimated_time=30,
            success_criteria=["test command"]
        )

    def test_repair_templates_initialization(self):
        """Test that repair templates are properly initialized"""
        templates = self.repair_engine.repair_templates

        # Check that basic templates exist
        self.assertIn("temp_cleanup", templates)
        self.assertIn("service_optimization", templates)
        self.assertIn("startup_optimization", templates)

        # Verify template structure
        temp_cleanup = templates["temp_cleanup"]
        self.assertIsInstance(temp_cleanup, RepairAction)
        self.assertTrue(len(temp_cleanup.commands) > 0)
        self.assertIsInstance(temp_cleanup.requires_admin, bool)

    @patch('subprocess.run')
    async def test_execute_command_success(self, mock_subprocess):
        """Test successful command execution"""
        # Mock successful subprocess result
        mock_result = Mock()
        mock_result.returncode = 0
        mock_result.stdout = "Command executed successfully"
        mock_result.stderr = ""
        mock_subprocess.return_value = mock_result

        result = await self.repair_engine._execute_command("echo test", False)

        self.assertTrue(result["success"])
        self.assertEqual(result["return_code"], 0)
        self.assertIn("Command executed successfully", result["stdout"])

    @patch('subprocess.run')
    async def test_execute_command_failure(self, mock_subprocess):
        """Test failed command execution"""
        # Mock failed subprocess result
        mock_result = Mock()
        mock_result.returncode = 1
        mock_result.stdout = ""
        mock_result.stderr = "Command failed"
        mock_subprocess.return_value = mock_result

        result = await self.repair_engine._execute_command("invalid_command", False)

        self.assertFalse(result["success"])
        self.assertEqual(result["return_code"], 1)
        self.assertIn("Command failed", result["stderr"])

    @patch('subprocess.run')
    async def test_execute_repair_success(self, mock_subprocess):
        """Test successful repair execution"""
        # Mock successful subprocess result
        mock_result = Mock()
        mock_result.returncode = 0
        mock_result.stdout = "test command executed"
        mock_result.stderr = ""
        mock_subprocess.return_value = mock_result

        progress_callback = Mock()
        result = await self.repair_engine.execute_repair(
            self.test_repair_action,
            progress_callback
        )

        self.assertTrue(result["success"])
        self.assertIn("repair_id", result)
        self.assertIn("duration", result)

        # Verify progress callback was called
        self.assertTrue(progress_callback.called)

    def test_check_success_criteria_with_criteria(self):
        """Test success criteria checking with specific criteria"""
        results = [
            {"stdout": "Operation completed successfully", "stderr": "", "success": True},
            {"stdout": "All tests passed", "stderr": "", "success": True}
        ]
        criteria = ["completed successfully", "tests passed"]

        success = self.repair_engine._check_success_criteria(results, criteria)
        self.assertTrue(success)

    def test_check_success_criteria_without_criteria(self):
        """Test success criteria checking without specific criteria"""
        results = [
            {"success": True},
            {"success": True}
        ]
        criteria = []

        success = self.repair_engine._check_success_criteria(results, criteria)
        self.assertTrue(success)

    def test_check_success_criteria_failure(self):
        """Test success criteria checking with failure"""
        results = [
            {"stdout": "Operation failed", "stderr": "Error occurred", "success": False}
        ]
        criteria = ["completed successfully"]

        success = self.repair_engine._check_success_criteria(results, criteria)
        self.assertFalse(success)

    async def test_repair_progress_tracking(self):
        """Test repair progress tracking functionality"""
        repair_id = "test_repair_123"

        # Test progress update
        await self.repair_engine._update_repair_progress(
            repair_id, "Starting repair", 0, "running"
        )

        # Since repair_id doesn't exist, it should log a warning
        # This tests the error handling in progress tracking

    @patch('subprocess.run')
    async def test_rollback_functionality(self, mock_subprocess):
        """Test repair rollback functionality"""
        # Create repair action with rollback commands
        repair_action = RepairAction(
            action_id="rollback_test",
            name="Rollback Test",
            description="Test rollback functionality",
            commands=["failing_command"],
            requires_admin=False,
            estimated_time=30,
            success_criteria=["success"],
            rollback_commands=["echo 'rolling back'"]
        )

        # Mock failing command followed by successful rollback
        mock_results = [
            Mock(returncode=1, stdout="", stderr="Command failed"),  # Failing command
            Mock(returncode=0, stdout="rolling back", stderr="")     # Successful rollback
        ]
        mock_subprocess.side_effect = mock_results

        repair_id = "rollback_test_123"

        # This should trigger rollback due to command failure
        with self.assertRaises(Exception):
            await self.repair_engine._execute_repair_action(
                repair_action, None, repair_id
            )

class TestProtectionEngine(unittest.TestCase):
    """Advanced tests for ProtectionEngine"""

    def setUp(self):
        """Set up test environment"""
        with patch('threading.Thread'):  # Prevent actual thread creation during tests
            self.protection_engine = ProtectionEngine()

    def test_protection_rules_initialization(self):
        """Test protection rules initialization"""
        rules = self.protection_engine.protection_rules

        self.assertIn("file_integrity", rules)
        self.assertIn("registry_protection", rules)

        # Check file integrity rules
        file_rules = rules["file_integrity"]
        self.assertIn("monitor_paths", file_rules)
        self.assertIn("check_interval", file_rules)
        self.assertIsInstance(file_rules["monitor_paths"], list)

    def test_threat_database_loading(self):
        """Test threat database loading"""
        threat_db = self.protection_engine.threat_database

        # Should have basic structure even if file doesn't exist
        self.assertIsInstance(threat_db, dict)

    @patch('os.path.exists')
    @patch('builtins.open')
    def test_threat_database_from_file(self, mock_open, mock_exists):
        """Test loading threat database from file"""
        mock_exists.return_value = True
        mock_file_content = {
            "malicious_hashes": ["abc123", "def456"],
            "suspicious_domains": ["malware.com"],
            "last_updated": "2023-01-01T00:00:00"
        }
        mock_open.return_value.__enter__.return_value.read.return_value = json.dumps(mock_file_content)

        with patch('threading.Thread'):
            engine = ProtectionEngine()

        # The threat database should contain the mocked data
        self.assertIsInstance(engine.threat_database, dict)

    def test_file_hash_calculation(self):
        """Test file hash calculation"""
        # Create a temporary file for testing
        with tempfile.NamedTemporaryFile(mode='w', delete=False) as temp_file:
            temp_file.write("test content for hashing")
            temp_file_path = temp_file.name

        try:
            file_hash = self.protection_engine._calculate_file_hash(temp_file_path)
            self.assertIsInstance(file_hash, str)
            self.assertEqual(len(file_hash), 64)  # SHA-256 hash length
        finally:
            os.unlink(temp_file_path)

    def test_file_hash_calculation_nonexistent_file(self):
        """Test file hash calculation for non-existent file"""
        file_hash = self.protection_engine._calculate_file_hash("nonexistent_file.txt")
        self.assertIsNone(file_hash)

: 